In [1]:
import sys
import numpy as np
import os
current_directory = os.getcwd()
relative_path1 = '../build/env_simulator'
relative_path2 = '../build/cilqr'
file_path1 = os.path.join(current_directory, relative_path1)
file_path2 = os.path.join(current_directory, relative_path2)
sys.path.append(file_path1)
sys.path.append(file_path2)
import pybind_env_simulator

relative_path3 = '../env_simulator'
file_path3 = os.path.join(current_directory, relative_path3)
sys.path.append(file_path3)
from py_lib import py_env_sim
from py_lib import cilqr_plot
import pybind_ilqr
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [2]:
def get_use_exp_list(l_conditions):
    use_exp_list = []
    for condition in l_conditions:
        use_exp = False
#         print(condition.use_exp_model_map)
        for item in condition.use_exp_model_map:
            if condition.use_exp_model_map[item] == True:
                use_exp = True
                break
        use_exp_list.append(use_exp)
    return use_exp_list

In [3]:
def show_motion_case(motion): 
    motion.execute()

    # plot 
    env = motion.get_env()
    planning_origin = motion.get_planning_origin()

    pts = env.get_all_center_points()
    obs = env.get_all_obstacle()
    planning_init = {}

    planning_init['x'] = planning_origin.position.x
    planning_init['y'] = planning_origin.position.y
    planning_init['theta'] = planning_origin.theta

    trajs = motion.get_init_trajectory()
    traj_list = []
    for traj in trajs:
        pt = {}
        pt['x'] = traj.position.x
        pt['y'] = traj.position.y
        pt['theta'] = traj.theta
        traj_list.append(pt)

    trajs_new = motion.get_new_trajectory()
    v_ref = motion.get_ref_v()
    a_ref = motion.get_ref_a()
    traj_new_list = []
    index = 0
    for traj in trajs_new:
        pt = {}
        pt['x'] = traj.position.x
        pt['y'] = traj.position.y
        pt['theta'] = traj.theta
        pt['v'] = traj.velocity
        pt['acc'] = traj.acceleration
        pt['ref_vel'] = v_ref[index]
        pt['ref_acc'] = a_ref[index]
        index = index + 1
        traj_new_list.append(pt)

    match_points = trajs = motion.get_match_point()
    l_conditions = motion.get_l_condition()
    # py_env_sim.plot_env(pts,obs)
    use_exp_list = get_use_exp_list(l_conditions)
    cilqr_plot.plot_cilqr_constraint(pts,obs,planning_init,traj_list, traj_new_list,use_exp_list)
#     cilqr_plot.plot_cilqr_env(pts,obs,planning_init, traj_list,traj_new_list,match_points)

In [ ]:
motion = pybind_ilqr.LateralLongitudinalMotion()
point_init = pybind_ilqr.PlanningPoint()
point_init.position.x = 10.0
point_init.position.y = 4.0
point_init.theta = 0.05
point_init.velocity = 25.0
point_init.acceleration  = 0.5

path = str(current_directory)+str("/../env_simulator/data/test_case_1_straight_lanes_cruise.json")
motion.init(path,point_init)

show_motion_case(motion)
